jupyter nbconvert 7_1_gcta_bivar.ipynb --to script


for i in {00..20};do python 7_1_gcta_bivar.py $i;done
for i in {21..40};do python 7_1_gcta_bivar.py $i;done
for i in {41..60};do python 7_1_gcta_bivar.py $i;done
for i in {61..80};do python 7_1_gcta_bivar.py $i;done
for i in {81..101};do python 7_1_gcta_bivar.py $i;done



python 7_1_gcta_bivar.py $WINDOW

for i in {00..101};do python 6_gcta_uni.py $i;done

for i in {00..10};do python 4_association.py $i;done


https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6292650/

In [1]:
import os
import sys

import logging
import re
import pathlib

import datetime

import numpy as np
import pandas as pd
from scipy.stats import pearsonr

import matplotlib.pyplot as plt

from pyplink import PyPlink

from basic_tools import *

In [2]:
log = logging.getLogger('logger')
log.setLevel(logging.DEBUG)

log_file_name=datetime.datetime.now().strftime('%Y%m%d_%H%M%S')+'.log'
log_file_path=data_out_gcta_path+log_file_name
fileHandler = logging.FileHandler(log_file_path)
streamHandler = logging.StreamHandler()

formatter = logging.Formatter(' %(asctime)s [%(levelname)s] %(lineno)d > %(message)s')
fileHandler.setFormatter(formatter)
streamHandler.setFormatter(formatter)

log.addHandler(fileHandler)
log.addHandler(streamHandler)

In [3]:
result_uni=pd.read_csv(data_out_gcta_path+'result_uni.tsv',sep='\t',index_col=0)

In [4]:
grm_path='data/genotype/4_merge/KCHIP_HLA_AA_SNP_1000G.grm'

In [5]:
result_uni.shape

(101, 10)

In [6]:
#i=0

In [7]:
#(pheno['pheno_x']!=-9).sum(),(pheno['pheno_y']!=-9).sum(),((pheno['pheno_x']!=-9) & (pheno['pheno_y']!=-9)).sum()

In [8]:
if 'ipykernel' in sys.argv[0]:
    ipykernel=True
    i=0
    #phenotype_name='height'
else:
    ipykernel=False
    i=int(sys.argv[1])  
    
log.info(i)

 2020-03-04 22:52:26,520 [INFO] 9 > 0


In [9]:
#pd.read_csv(data_out_gcta_path+phenotype_name1+'-'+phenotype_name2+'.phe')

In [15]:
for j in range(i+1,len(result_uni)):
    phenotype_name1=result_uni.iloc[i].name
    phenotype_name2=result_uni.iloc[j].name
    
    if phenotype_name1=='age' or phenotype_name2=='age':
        continue
    
    if os.path.exists(data_out_gcta_path+phenotype_name1+'-'+phenotype_name2+'.HEreg') or os.path.exists(data_out_gcta_path+phenotype_name2+'-'+phenotype_name1+'.HEreg'):
        continue



    pheno1=pd.read_csv(data_out_pheno_path+phenotype_name1+'.phe',sep='\t',header=None,names=['FID','IID','pheno'])
    pheno2=pd.read_csv(data_out_pheno_path+phenotype_name2+'.phe',sep='\t',header=None,names=['FID','IID','pheno'])
    pheno=pheno1.merge(right=pheno2,left_on=['FID','IID'],right_on=['FID','IID'])

    pheno_filter=pheno[(pheno['pheno_x']!=-9) & (pheno['pheno_y']!=-9)]
    pheno_filter.to_csv(data_out_gcta_path+phenotype_name1+'-'+phenotype_name2+'.phe',sep='\t',index=None,header=None)

    log.info("phenotype_name1: {}, phenotype_name2:{}".format(phenotype_name1,phenotype_name2))
    log.info('pheno1 mising {}'.format((pheno['pheno_x']!=-9).sum()))
    log.info('pheno2 mising {}'.format((pheno['pheno_y']!=-9).sum()))

    if os.path.exists(data_out_gcta_path+phenotype_name1+'-'+phenotype_name2+'.HEreg'):
        log.info(data_out_gcta_path+phenotype_name1+'-'+phenotype_name2+'.HEreg exists')
        break
    
    log.info("#########################################  Run GCTA  #########################################")
    #Run omnibus association test
    command='gcta64 --HEreg-bivar 1 2 --grm {} --pheno {} --out {} --thread-num 10'.format(grm_path,
                                                                                 data_out_gcta_path+phenotype_name1+'-'+phenotype_name2+'.phe',
                                                                                 data_out_gcta_path+phenotype_name1+'-'+phenotype_name2,                                         
                                                                                )
    log.info(command)
    stdout,stderr=run_subprocess(command,dry=False)
    log.info(stdout)
    log.error(stderr)          

 2020-03-03 21:31:19,299 [INFO] 13 > phenotype_name1: ALP, phenotype_name2:FEV_over_FVC_predicted
 2020-03-03 21:31:19,299 [INFO] 13 > phenotype_name1: ALP, phenotype_name2:FEV_over_FVC_predicted
 2020-03-03 21:31:19,306 [INFO] 14 > pheno1 mising 59469
 2020-03-03 21:31:19,306 [INFO] 14 > pheno1 mising 59469
 2020-03-03 21:31:19,309 [INFO] 15 > pheno2 mising 7440
 2020-03-03 21:31:19,309 [INFO] 15 > pheno2 mising 7440
 2020-03-03 21:31:19,310 [INFO] 17 > #########################################  Run GCTA  #########################################
 2020-03-03 21:31:19,310 [INFO] 17 > #########################################  Run GCTA  #########################################
 2020-03-03 21:31:19,311 [INFO] 23 > gcta64 --HEreg-bivar 1 2 --grm data/genotype/4_merge/KCHIP_HLA_AA_SNP_1000G.grm --pheno data/out_gcta/ALP-FEV_over_FVC_predicted.phe --out data/out_gcta/ALP-FEV_over_FVC_predicted --thread-num 1
 2020-03-03 21:31:19,311 [INFO] 23 > gcta64 --HEreg-bivar 1 2 --grm data/genotype/

------RUN-----
gcta64 --HEreg-bivar 1 2 --grm data/genotype/4_merge/KCHIP_HLA_AA_SNP_1000G.grm --pheno data/out_gcta/ALP-FEV_over_FVC_predicted.phe --out data/out_gcta/ALP-FEV_over_FVC_predicted --thread-num 1


 2020-03-03 21:31:19,649 [INFO] 25 > *******************************************************************
* Genome-wide Complex Trait Analysis (GCTA)
* version 1.92.2 beta Linux
* (C) 2010-2019, The University of Queensland
* Please report bugs to: Jian Yang <jian.yang@uq.edu.au>
*******************************************************************
Analysis started at 12:31:19 UTC on Tue Mar 03 2020.
Hostname: localhost.localdomain

Accepted options:
--HEreg-bivar 1 2
--grm data/genotype/4_merge/KCHIP_HLA_AA_SNP_1000G.grm
--pheno data/out_gcta/ALP-FEV_over_FVC_predicted.phe
--out data/out_gcta/ALP-FEV_over_FVC_predicted
--thread-num 1

Note: This is a multi-thread program. You could specify the number of threads by the --thread-num option to speed up the computation if there are multiple processors in your machine.

Reading IDs of the GRM from [data/genotype/4_merge/KCHIP_HLA_AA_SNP_1000G.grm.grm.id].
125673 IDs read from [data/genotype/4_merge/KCHIP_HLA_AA_SNP_1000G.grm.grm.id].
Reading 

------ERROR-----

------OUTPUT-----
*******************************************************************
* Genome-wide Complex Trait Analysis (GCTA)
* version 1.92.2 beta Linux
* (C) 2010-2019, The University of Queensland
* Please report bugs to: Jian Yang <jian.yang@uq.edu.au>
*******************************************************************
Analysis started at 12:31:19 UTC on Tue Mar 03 2020.
Hostname: localhost.localdomain

Accepted options:
--HEreg-bivar 1 2
--grm data/genotype/4_merge/KCHIP_HLA_AA_SNP_1000G.grm
--pheno data/out_gcta/ALP-FEV_over_FVC_predicted.phe
--out data/out_gcta/ALP-FEV_over_FVC_predicted
--thread-num 1

Note: This is a multi-thread program. You could specify the number of threads by the --thread-num option to speed up the computation if there are multiple processors in your machine.

Reading IDs of the GRM from [data/genotype/4_merge/KCHIP_HLA_AA_SNP_1000G.grm.grm.id].
125673 IDs read from [data/genotype/4_merge/KCHIP_HLA_AA_SNP_1000G.grm.grm.id].
Reading p

 2020-03-03 21:31:20,559 [INFO] 13 > phenotype_name1: ALP, phenotype_name2:FEV_predicted
 2020-03-03 21:31:20,559 [INFO] 13 > phenotype_name1: ALP, phenotype_name2:FEV_predicted
 2020-03-03 21:31:20,562 [INFO] 14 > pheno1 mising 59469
 2020-03-03 21:31:20,562 [INFO] 14 > pheno1 mising 59469
 2020-03-03 21:31:20,564 [INFO] 15 > pheno2 mising 7440
 2020-03-03 21:31:20,564 [INFO] 15 > pheno2 mising 7440
 2020-03-03 21:31:20,565 [INFO] 17 > #########################################  Run GCTA  #########################################
 2020-03-03 21:31:20,565 [INFO] 17 > #########################################  Run GCTA  #########################################
 2020-03-03 21:31:20,567 [INFO] 23 > gcta64 --HEreg-bivar 1 2 --grm data/genotype/4_merge/KCHIP_HLA_AA_SNP_1000G.grm --pheno data/out_gcta/ALP-FEV_predicted.phe --out data/out_gcta/ALP-FEV_predicted --thread-num 1
 2020-03-03 21:31:20,567 [INFO] 23 > gcta64 --HEreg-bivar 1 2 --grm data/genotype/4_merge/KCHIP_HLA_AA_SNP_1000G.grm -

------RUN-----
gcta64 --HEreg-bivar 1 2 --grm data/genotype/4_merge/KCHIP_HLA_AA_SNP_1000G.grm --pheno data/out_gcta/ALP-FEV_predicted.phe --out data/out_gcta/ALP-FEV_predicted --thread-num 1


KeyboardInterrupt: 